# Compiling and Submitting Circuits onto Hilbert via Cirq


<a target="_blank" href="https://colab.research.google.com/github/SupertechLabs/client-superstaq/blob/main/docs/source/optimizations/cq/cq_compile_css.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" align="left"/>
</a>
<a target="_blank" href="https://mybinder.org/v2/gh/SupertechLabs/client-superstaq/HEAD?labpath=docs/source/optimizations/cq/cq_compile_css.ipynb">
  <img src="https://mybinder.org/badge_logo.svg" alt="Launch Binder" align="left" style="margin-left: 10px;"/>
</a>

## Import Requirements

This tutorial will showcase how to compile and submit a circuit onto Infleqtion's hardware, Hilbert, using the ```cirq-superstaq``` client. You can try it out by running ```pip install cirq-superstaq```:

In [ ]:
%pip install cirq-superstaq

In [1]:
# Required imports
import cirq
import cirq_superstaq as css

# Optional imports
import os  # used if setting a token as an environment variable

To interface Superstaq via Cirq, we must first instantiate a service provider in ```cirq-superstaq``` with ```Service()```. We then supply a Superstaq API key (which you can get from https://superstaq.super.tech.) by either providing the API key as an argument of Service, i.e. : ```css.Service(api_key="token")``` , or by setting it as an environment variable. (see more details [here](https://docs-superstaq.readthedocs.io/en/stable/get_started/basics/basics_css.html#Set-up-access-to-Superstaq's-API)) 

In [2]:
service = css.Service()

## Create a Circuit

First, we will create an example Cirq circuit that we can then compile and submit onto Hilbert

In [3]:
qubits = cirq.LineQubit.range(16)
circuit1 = cirq.Circuit(cirq.H(qubits[0]), cirq.CNOT(qubits[0], qubits[1]), cirq.measure(qubits[0]))
print(circuit1)

0: ───H───@───M───
          │
1: ───────X───────


## Single Circuit Compilation

In [4]:
compiler_output = service.cq_compile(circuit1)

In [5]:
print(compiler_output.circuit)

(0, 0): ───RGate(0.5π, -0.5π)───@───RGate(-0.25π, -0.5π)───────RGate(0.25π, -0.5π)───────M('q(0)')───
           │                    │   │                          │
(0, 1): ───#2───────────────────@───#2─────────────────────Z───#2────────────────────Z───────────────


If you would like to compile (or submit) on a different number of qubits, this can be done via the ```grid_shape``` option. This simply sets the shape of the rectangular qubit grid. However, specifying a grid that is incompatable with Hilbert's current capabilities will result in an error. 

In [6]:
example_circuit = cirq.Circuit(cirq.H.on_each(*qubits))
new_compiler_output = service.cq_compile(example_circuit, options={"grid_shape":(4,6)})
print(new_compiler_output.circuit)

(0, 0): ───RGate(0.5π, -0.5π)───Z───
           │
(0, 1): ───#2───────────────────Z───
           │
(0, 2): ───#3───────────────────Z───
           │
(0, 3): ───#4───────────────────Z───
           │
(0, 4): ───#5───────────────────Z───
           │
(0, 5): ───#6───────────────────Z───
           │
(1, 0): ───#7───────────────────Z───
           │
(1, 1): ───#8───────────────────Z───
           │
(1, 2): ───#9───────────────────Z───
           │
(1, 3): ───#10──────────────────Z───
           │
(1, 4): ───#11──────────────────Z───
           │
(1, 5): ───#12──────────────────Z───
           │
(2, 0): ───#13──────────────────Z───
           │
(2, 1): ───#14──────────────────Z───
           │
(2, 2): ───#15──────────────────Z───
           │
(2, 3): ───#16──────────────────Z───


## Single Circuit Submission

The code below will submitt he circuit to the noiseless Hilbert simulator. If you would like to run on Hilbert, change the "target" argument in job_css from "cq_hilbert_simulator" to "cq_hilbert_qpu". It is recommended to first submit to the simulator for testing to ensure your code runs before submitting to Hilbert. 

In [7]:
job_css = service.create_job(circuit=circuit1, repetitions=100, target="cq_hilbert_simulator")
result_css = job_css.counts()
print(f"Counts from cirq-superstaq submission: {result_css}")

Counts from cirq-superstaq submission: {'1': 55, '0': 45}


## Multiple circuit compilation

In [8]:
circuit2 = cirq.Circuit(cirq.H(qubits[0]), cirq.measure(qubits[0]))
print(circuit2)

0: ───H───M───


In [9]:
compiler_output = service.cq_compile([circuit1, circuit2])

In [10]:
print(compiler_output.circuits[0])
print()
print()
print(compiler_output.circuits[1])

(0, 0): ───RGate(0.5π, -0.5π)───@───RGate(-0.25π, -0.5π)───────RGate(0.25π, -0.5π)───────M('q(0)')───
           │                    │   │                          │
(0, 1): ───#2───────────────────@───#2─────────────────────Z───#2────────────────────Z───────────────


(0, 0): ───RGate(0.5π, -0.5π)───M('q(0)')───
